In [1]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
import pandas as pd
import numpy as py
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score,log_loss,accuracy_score,precision_score,recall_score,f1_score

In [2]:
xgb=pd.read_csv('proba_XG1.csv')
rf=pd.read_csv('proba_RF1.csv')
sv=pd.read_csv('proba_LR1.csv')
#to=pd.read_csv('to (4).csv')
perf=pd.read_csv('Thesis/Performance1.csv')# Performance CHANGE make sure when import there is an additional 7 days from the beginning for test results
xgb=xgb.iloc[:,1:].dropna()
rf=rf.iloc[:,1:].dropna()
sv=sv.iloc[:,1:].dropna()
y_r=perf.iloc[252:,1:].dropna()

In [3]:
RF=[]
XGB=[]
SVM=[]
X_r=[]
for i in range(93):
    XGB.append(py.concatenate(py.array(xgb.iloc[:,42*i:42*(i+1)]),axis=None))
    RF.append(py.concatenate(py.array(rf.iloc[:,42*i:42*(i+1)]),axis=None))
    SVM.append(py.concatenate(py.array(sv.iloc[:,42*i:42*(i+1)]),axis=None))
    X_r.append(pd.DataFrame([XGB[i],RF[i],SVM[i]]).T)

In [4]:
# Split Recent Training- WALK FORWARD
# Might need to slice data for the slack- update later
def split_X_y_ticker(j):
    for i in range(84):
        #Split train,test
        X=X_r[j].iloc[42*i:252+42*(i+1),:]
        y=py.array(y_r.iloc[42*i:252+42*(i+1),j])
        scale=StandardScaler()
        X=scale.fit_transform(X)
        X_tr=X[:-42]
        y_tr=y[:-42]
        X_t=X[-42:]
        y_t=y[-42:]
        #Split train,val
        X_train=[]
        X_val=[]
        y_train=[]
        y_val=[]
        for a in range(3):
            X_train.append(X_tr[:126+42*(a)])#Variables to be change then change shape 
            y_train.append(y_tr[:126+42*(a)])
            X_val.append(X_tr[126+42*(a):126+42*(a+1)])
            y_val.append(y_tr[126+42*(a):126+42*(a+1)])
        y_t=y_t
        X_t=X_t
        yield X_train,X_val,y_train,y_val,y_t,X_t,X_tr,y_tr

In [5]:
def compile_for_cross_sectional_learning():
    X_train=[]
    X_val=[]
    y_train=[]
    y_val=[]
    y_t=[]
    X_t=[]
    X_tr=[]
    y_tr=[]
    for i in range(93):
        X_train_tick=[]
        X_val_tick=[]
        y_train_tick=[]
        y_val_tick=[]
        y_t_tick=[]
        X_t_tick=[]
        X_tr_tick=[]
        y_tr_tick=[]
        a=split_X_y_ticker(i)
        for X_train_slide,X_val_slide,y_train_slide,y_val_slide,y_t_slide,X_t_slide,X_tr_slide,y_tr_slide in a:
            X_train_tick.append(X_train_slide)
            X_val_tick.append(X_val_slide)
            y_train_tick.append(y_train_slide)
            y_val_tick.append(y_val_slide)
            y_t_tick.append(y_t_slide)
            X_t_tick.append(X_t_slide)
            X_tr_tick.append(X_tr_slide)
            y_tr_tick.append(y_tr_slide)
        X_train.append(X_train_tick)
        X_val.append(X_val_tick)
        y_train.append(y_train_tick)
        y_val.append(y_val_tick)
        y_t.append(y_t_tick)
        X_t.append(X_t_tick)
        X_tr.append(X_tr_tick)
        y_tr.append(y_tr_tick)
    X_train=py.stack([X_train[i] for i in range(93)],axis=-1)
    y_train=py.stack([y_train[i] for i in range(93)],axis=-1)
    X_val=py.stack([X_val[i] for i in range(93)],axis=-1)
    y_val=py.stack([y_val[i] for i in range(93)],axis=-1)
    X_t=py.stack([X_t[i] for i in range(93)],axis=-1)
    y_t=py.stack([y_t[i] for i in range(93)],axis=-1)
    X_tr=py.stack([X_tr[i] for i in range(93)],axis=-1)
    y_tr=py.stack([y_tr[i] for i in range(93)],axis=-1)
    return X_train,X_val,y_train,y_val,y_t,X_t,X_tr,y_tr

In [6]:
def walk_forward_val(model):#model already defined
    Mean_Score_SV=[]
    prec=[]
    recall=[]
    f1=[]
    X_train_r,X_val_r,y_train_r,y_val_r,y_t_r,X_t_r,X_tr_r,y_tr_r=compile_for_cross_sectional_learning()
    for j in range(len(X_train_r)):
        for i in range(len(X_train_r[0])):
            X_train=py.concatenate(X_train_r[j][i])
            y_train=py.concatenate(y_train_r[j][i])
            X_val=X_val_r[j][i].reshape(-1,3)
            y_val=y_val_r[j][i].reshape(-1,)
            model.fit(X_train,y_train,epochs=30,verbose=0)
            y_pred_val=model.predict(X_val)
            Mean_Score_SV.append(accuracy_score(y_val,y_pred_val))
            prec.append(precision_score(y_val,y_pred_val)) 
            recall.append(recall_score(y_val,y_pred_val)) #Specificity
            f1.append(f1_score(y_val,y_pred_val))
    print(py.mean(Mean_Score_SV))
    print(' ')
    print(py.mean(prec))
    print(' ')
    print(py.mean(recall))
    print(' ')
    print(py.mean(f1))
    return py.mean(Mean_Score_SV),py.mean(prec),py.mean(recall),py.mean(f1)

In [7]:
def ANN_construction(layers,activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=3))
            model.add(Activation(activation))
            model.add(Dropout(0.1))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.1))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) 
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model  

In [8]:
def ANN_grid_tuning(params):
    best_score=0
    for param in params :
        model=ANN_construction(param[0],param[1])
        mod=KerasClassifier(build_fn= lambda: model,verbose=0)
        acc,prec,recall, f1= walk_forward_val(mod)
        if acc > best_score:
            best_score=acc
            sensi=recall
            preci=prec
            best_param=param
    return best_score,sensi,preci,best_param

In [10]:
def ANN_stacked_predict(param):
    acc=[]
    y_pred_all=[]
    model=ANN_construction(param[0],param[1])
    mod=KerasClassifier(build_fn=lambda:model,verbose=0)
    X_train_r,X_val_r,y_train_r,y_val_r,y_t_r,X_t_r,X_tr_r,y_tr_r=compile_for_cross_sectional_learning()
    for i in range(len(X_train_r)):
        X_t=X_t_r[i].reshape(-1,3)
        y_t=y_t_r[i].reshape(-1,)
        X_tr=X_tr_r[i].reshape(-1,3)
        y_tr=y_tr_r[i].reshape(-1,)
        mod.fit(X_tr,y_tr,epochs=100,verbose=0)
        y_pred=mod.predict_proba(X_t)
        y_score=mod.predict(X_t)
        y_pred_all.append(y_pred[:,-1])
        acc.append(accuracy_score(y_t,y_score))
    return y_pred_all,acc

In [11]:
paramsANN=[[[32],'relu'],
           [[25],'sigmoid'],
           [[32,16],'relu'],
           [[12],'sigmoid']]

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
best_score,sensi,preci,best_param=ANN_grid_tuning(paramsANN)

In [ ]:
print(best_score,preci,sensi,best_param)

In [15]:
y_pred_all,log=ANN_stacked_predict(paramsANN[0])

In [17]:
pd.DataFrame(log).to_csv('1ANN_acc.csv')